<a href="https://colab.research.google.com/github/pastrop/kaggle/blob/master/Llama_index.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install llama-index

In [ ]:
%%capture
!pip install transformers

In [ ]:
%%capture
!pip install sentence_transformers

In [ ]:
%%capture
!pip install llama-cpp-python

In [ ]:
%%capture
!pip install accelerate

In [ ]:
!pip install nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 45.6 MB/s eta 0:00:00


In [ ]:
import torch
import pandas as pd
from llama_index import Document
from llama_index.node_parser import SimpleNodeParser
from llama_index import VectorStoreIndex
from llama_index.llms import HuggingFaceLLM

In [ ]:
from llama_index.llms import HuggingFaceLLM
from llama_index import VectorStoreIndex, ServiceContext
from llama_index.prompts import PromptTemplate
import accelerate

In [ ]:
torch.cuda.is_available()

True

In [ ]:
system_prompt = """<|SYSTEM|># StableLM Tuned (Alpha version)
- StableLM is a helpful and harmless open-source AI language model developed by StabilityAI.
- StableLM is excited to be able to help the user, but will refuse to do anything that could be considered harmful to the user.
- StableLM is more than just an information source, StableLM is also able to write poetry, short stories, and make jokes.
- StableLM will refuse to participate in anything that could harm a human.
"""

# This will wrap the default prompts that are internal to llama-index
query_wrapper_prompt = PromptTemplate("<|USER|>{query_str}<|ASSISTANT|>")

llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.7, "do_sample": False},
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="StabilityAI/stablelm-tuned-alpha-3b",
    model_name="StabilityAI/stablelm-tuned-alpha-3b",
    device_map="auto",
    stopping_ids=[50278, 50279, 50277, 1, 0],
    tokenizer_kwargs={"max_length": 4096},
    # uncomment this if using CUDA to reduce memory usage
    model_kwargs={"torch_dtype": torch.float16}
)

In [ ]:
%%capture
service_context = ServiceContext.from_defaults(chunk_size=1024, llm=llm, embed_model = 'local')

In [ ]:
# Dataframe processing
df_acct = pd.read_csv('accnt_issues.csv')

In [ ]:
#input cleanup
def text_input(file = 'alpha_test.csv'):
  df = pd.read_csv(file)
  df_clean = df[df['text'].apply(lambda x: isinstance(x, str))]
  texts = [item.replace("\t", " ") for item in df_clean['text']]
  return texts

In [ ]:
texts = text_input('accnt_issues.csv')

In [ ]:
#test cell
texts[:10]

In [ ]:
#document list
documents = [Document(text=t) for t in texts]

In [ ]:
%%capture
parser = SimpleNodeParser.from_defaults()
nodes = parser.get_nodes_from_documents(documents)

In [ ]:
index = VectorStoreIndex.from_documents(documents,service_context=service_context)

In [ ]:
query_engine = index.as_query_engine()
#response = query_engine.query("What did the author do growing up?")

In [ ]:
query_engine.query("what was this document set is about?")

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Response(response='Based on the context information provided, it seems that the document you are referring to is likely a game or achievement that was released for the Xbox platform. The game or achievement may have been a free downloadable game or a paid downloadable game.\n\nThe document you are referring to is likely a game or achievement that was released for the Xbox platform. The game or achievement may have been a free downloadable game or a paid downloadable game.\n\nIt is unclear from the context whether the document you are referring to is about a specific game or achievement or whether it is about a general document. If it is about a specific game or achievement, then the document you are referring to is likely a game or achievement that was released for the Xbox platform.\n\nIf it is about a general document, then the document you are referring to is likely a game or achievement that was released for the Xbox platform. The document you are referring to is likely a game or a

In [ ]:
query_engine.query("is it possible to connect Xbox to Epic?")

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Response(response='Yes, it is possible to connect your Xbox account to your Epic Games account. However, it is not possible to link your Xbox account to your Epic Games account directly. Instead, you will need to link your Xbox account to your Epic Games account using the email address provided in the context information.\n\nTo link your Xbox account to your Epic Games account, you will need to follow these steps:\n\n1. Log in to your Xbox account using the email address provided in the context information.\n2. Click on the "Settings" tab in the top left corner of the screen.\n3. Select "Accounts" from the drop-down menu.\n4. Click on "Add Account" and follow the prompts to create a new account.\n5. Once the account is created, you will see a list of your Xbox accounts. Select the account you want to link to.\n6. Click on the "Link" button to add the account to your account.\n\nOnce you have linked your Xbox account to your Epic Games account, you will be able to access your Xbox live 

In [ ]:
# beer dataset

In [ ]:
# Dataframe processing
df = pd.read_csv('alpha_test.csv')

In [ ]:
df.head()

In [ ]:
texts_beer = text_input()

In [ ]:
len(texts_beer)

4998

In [ ]:
%%capture
documents_beer = [Document(text=t) for t in texts_beer]

parser = SimpleNodeParser.from_defaults()
nodes = parser.get_nodes_from_documents(documents_beer)

index_beer = VectorStoreIndex.from_documents(documents_beer,service_context=service_context)
query_engine_beer = index_beer.as_query_engine()

In [ ]:
query_engine_beer.query("based on the set of documents provided what people say about IPA?")

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Response(response='Based on the context information, it is unclear what people say about the IPA. However, based on the information provided, it can be inferred that the IPA is a highly hoppy and flavorful beer with a smooth, dry finish. The IPA has a light, delicate flavor with a hint of bitterness, which is typical of American West Coast IPA beers. The IPA is also light bodied with a good amount of hop bitterness, which is typical of American IPAs. The IPA is also a highly drinkable, with a smooth, dry finish. Overall, it seems that the IPA is a well-crafted and highly regarded beer with a unique flavor profile.', source_nodes=[NodeWithScore(node=TextNode(id_='9e808a68-f8ad-46a1-b5ed-46c2f7675a57', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='f2e6226f-c730-40bc-b823-8c791516bdd5', node_type=None, metadata={}, hash='824c270e86fed94409d16c0088802c03fdf9233e02502311712

In [ ]:
query_engine_beer.query("based on the document set provided what beers people like the best?")

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Response(response='Based on the context information provided, I would recommend the following beers:\n\n1. Bluegrass Brewing Co. American Pale Ale\n2. Willamette Hop\n3. Centennial and Willamette Hop\n4. Centennial and Willamette Hop\n5. Centennial and Willamette Hop\n6. Centennial and Willamette Hop\n7. Centennial and Willamette Hop\n8. Centennial and Willamette Hop\n9. Centennial and Willamette Hop\n10. Centennial and Willamette Hop\n\nIt is important to note that this is not an exhaustive list and there are many other great beers out there, so it is worth trying a few to see what you like.', source_nodes=[NodeWithScore(node=TextNode(id_='295d18ab-5a04-4f46-81da-9c622ad26728', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='eb23560a-28f1-4226-9323-bce3a252b193', node_type=None, metadata={}, hash='a0d1f7d0e4c74cc43c3248b2a484e87ee47505b0e2821291479b27a253d08989')}, hash

In [ ]:
query_engine_beer.query("based on the document set provided what beers flavors are the most popular?")

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Response(response='Based on the context information, the most popular beer flavors are:\n\n1. American Pale Ale\n2. Bluegrass Brewing Co. American Pale Ale\n3. Willamette Hop\n4. Centennial and Willamette Hop\n5. Centennial and Willamette Hop\n6. Centennial and Willamette Hop\n7. Centennial and Willamette Hop\n8. Centennial and Willamette Hop\n9. Centennial and Willamette Hop\n10. Centennial and Willamette Hop\n\nIt is important to note that this is not an exhaustive list and there may be other beer flavors that are more popular than others. Additionally, the popularity of certain beers can change over time and may not be reflected in the provided document.', source_nodes=[NodeWithScore(node=TextNode(id_='bc2c39d5-417a-4ee8-9865-f84280c157e4', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='a67e8540-8500-451f-93cb-dbc631ec83a0', node_type=None, metadata={}, hash='6af9814

In [ ]:
#experimenting with the query indices

In [ ]:
nodes_beer = parser.get_nodes_from_documents(documents_beer)

In [ ]:
index_beer_other = VectorStoreIndex(nodes_beer, service_context=service_context)
#keyword_index = SimpleKeywordTableIndex(nodes_beer, service_context=service_context)

In [ ]:
query_engine_beer_other = index_beer_other.as_query_engine()

In [ ]:
query_engine_beer_other.query("based on the document set provided, what beers flavors are the most popular?")

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Response(response='Based on the context information, the most popular beer flavors are:\n\n1. American Pale Ale\n2. Bluegrass Brewing Co. American Pale Ale\n3. Willamette Hop\n4. Centennial and Willamette Hop\n5. Centennial and Willamette Hop\n6. Bluegrass Brewing Co. American Pale Ale\n7. American Pale Ale\n8. Centennial and Willamette Hop\n9. Bluegrass Brewing Co. American Pale Ale\n10. Willamette Hop\n\nNote: The popularity of these beers can vary depending on location and time of year.', source_nodes=[NodeWithScore(node=TextNode(id_='47966790-64e1-4db8-9471-1a9cb30e8e0e', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='a67e8540-8500-451f-93cb-dbc631ec83a0', node_type=None, metadata={}, hash='6af98142e133bacf037b0f5bfb7022706bea39caaffb1d3ae4481315fc92d410')}, hash='6af98142e133bacf037b0f5bfb7022706bea39caaffb1d3ae4481315fc92d410', text='Clear red colored ale with a g

In [ ]:
#pandas query engine (don't wor with the current model)
from llama_index.query_engine import PandasQueryEngine

In [ ]:
query_engine = PandasQueryEngine(df=df['text'], verbose=True, service_context=service_context)

In [ ]:
response = query_engine.query(
    "are their any beer related documents?",
)

In [ ]:
#query filter investigation
#from llama_index.vector_stores.types import ExactMatchFilter, MetadataFilters

query_engine_custom = index_beer.as_query_engine(
    similarity_top_k=2,
    vector_store_query_mode="default",
    #filters=MetadataFilters(
        #filters=[
            #ExactMatchFilter(key="name", value="paul graham"),
        #]
    #),
    alpha=None,
    doc_ids=None,
)

In [ ]:
response = query_engine_custom.query(" based on the documens provided what beers people like the most?")

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


In [ ]:
print(response)

Based on the context information, I would say that the beer that people like the most is likely to be Centennial and Willamette Hop. Centennial and Willamette Hop are both strong and creamy beers that have a lot of smoky and meaty flavor to them. They are also both located in downtown Louisville and have a rich amber color, creamy head, and good lacing in the glass. Additionally, they are both medium bodied and have a creamy mouthfeel, which is a nice characteristic. Overall, these beers are interesting and enjoyable to drink, and I would recommend them to others.


In [ ]:
summary = query_engine_custom.query(" based on the documents provided summarize opinions about IPA?")

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


In [ ]:
print(summary)

Based on the context information, it is unclear whether or not the documents provided opinions about the IPA. However, based on the information provided, it can be inferred that the IPA is a highly hoppy and flavorful beer with a smooth, dry finish. The amber color and sedimentation of the beer suggest that it may have been brewed with a lot of hops, which gives it a unique flavor profile. The IPA is also described as having a light, hoppy flavor with a tinge of malt and citrus, which could be due to the use of hops in the brewing process. Overall, the documents provide a good summary of the beer's flavor profile and characteristics.
